# Import Dependencies

In [1]:
from datagen_customer import Customer, Headers
from demographics import make_cities, make_age_gender_dict
from faker import Faker
import pandas as pd
import numpy as np
import datetime
import random
import google.auth
from google.cloud import storage
import os
import pandas_gbq
from os import listdir
from os.path import isfile, join
from datetime import date
import faker
import profile_weights
from datagen_transaction import T_Customer, create_header
import os
import multiprocessing as mp
#print("Number of processors: ", mp.cpu_count())
from google.cloud import storage

In [2]:
#service account needs write access into bq project
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/leighaj/Service Accounts/looker-private-demo-d4bcf655c1aa.json'

credentials = credentials, project = google.auth.default(
    scopes=[
        "https://www.googleapis.com/auth/bigquery"
    ]
)

In [15]:
#set random seed for faker
fake = Faker()
Faker.seed(44)

# Create New Customers
### Skip if you already have a customers.csv

In [ ]:
#create 100,000 customers which are saved to file
for _ in range(100000):
    Customer(fake = fake, filename = 'customer_test.csv')

In [ ]:
#assign a traffic source
def create_traffic_source(profile):
    traffic_sources = ['Mailer','Email','Call Center','Website','Third Party Advertisement']
    if 'young_adults' in profile:
        #more likely to be third party adverisement or website
        traffic_sources = traffic_sources + [traffic_sources[4],traffic_sources[4],traffic_sources[4
                                    ],traffic_sources[3],traffic_sources[3]]
    elif 'urban' in profile:
        traffic_sources = traffic_sources + [traffic_sources[1],traffic_sources[2],traffic_sources[2
                        ],traffic_sources[3],traffic_sources[3]]
    else:
        traffic_sources = traffic_sources + [traffic_sources[1],traffic_sources[1],traffic_sources[1
                ],traffic_sources[2],traffic_sources[3]]
    
    return np.random.choice(traffic_sources)

In [ ]:
#read in csv as a dataframe
clients = pd.read_csv('customers.csv','|',names=['ssn', 'cc_num', 'first', 'last', 'gender', 'street', \
                    'city', 'state', 'zip', 'lat', 'long', 'city_pop', \
                    'job', 'dob', 'acct_num', 'profile'])

In [ ]:
clients.head()

In [ ]:
## already exists
#ssn
#first_name
#last_name
#gender
#street
#address
#job
#profile
#dob 

## need to create
#age - create in database
#district_id - keep as null
#client_id - keep as null

#client_id add row number 
clients = clients.reset_index()
clients['disp_id'] = clients['index'].apply(lambda x: x + 13661)
clients['card_id'] = clients['index'].apply(lambda x: x + 1462)

#traffic_source
clients['traffic_source'] = clients['profile'].apply(lambda x: create_traffic_source(x))

In [ ]:
clients = clients.sample(frac=1)
clients = clients.drop(['index'], axis=1)
clients = clients.reset_index()

clients['card_type'] = None
clients['card_start'] = None
clients['card_end'] = None

In [ ]:
#Give 70% of young adults junior card, 20% junior2 card and 6% classic, 3% classic2 and rest gold
length = len(clients)

clients.loc[clients['profile'].str.contains('young'),'card_type'] = clients.loc[
    clients['profile'].str.contains('young')].apply(lambda x: 'Junior' if int(x['index']) < .7*length 
            else 'Junior2' if int(x['index']) <.9*length 
            else 'Classic' if int(x['index']) <.96*length
            else 'Classic2' if int(x['index']) <.99*length else 'Gold', axis=1)

In [ ]:
#Give 50% of classic, 35% classic2, 15% gold 
#length = len(clients.loc[clients['profile'].str.contains('50up')])

clients.loc[clients['profile'].str.contains('50up'),'card_type'] = clients.loc[
    clients['profile'].str.contains('50up')].apply(lambda x: 'Classic' if int(x['index']) < .5*length 
            else 'Classic2' if int(x['index']) <.35*length
            else 'Gold', axis=1)

In [ ]:
#Give 60% of classic, 30% classic2, 10% gold 
#length = len(clients.loc[clients['profile'].str.contains('2550')])

clients.loc[clients['profile'].str.contains('2550'),'card_type'] = clients.loc[
    clients['profile'].str.contains('2550')].apply(lambda x: 'Classic' if int(x['index']) < .6*length 
            else 'Classic2' if int(x['index']) <.9*length 
            else 'Gold', axis=1)

In [ ]:
# randomly chose a start date that is after november 2015 more than 15 years after their birthday, and before end of this year

clients['card_start'] = clients['dob'].apply(lambda x: 
        datetime.timedelta(days=random.randint(max(15*365,(pd.to_datetime('2015-11-01')-pd.to_datetime(
                        x)).days), (pd.to_datetime(
        '2020-12-31')-pd.to_datetime(x)).days)) + pd.to_datetime(x))

In [ ]:
# for junior cards - take 80% of cards and create an end date and a new card add date
if 'index' in clients.columns:
    clients = clients.drop(['index'], axis=1)
results = clients.loc[clients['card_type'] == 'Junior'].sample(frac=.9).reset_index()


#randomly generate a normal distribution of time they convert to next card, and the time they take to cancel previous
days_to_add = list(np.random.normal(365, 150, len(results)))
days_to_cancel = list(np.random.normal(30*3, 25, len(results)))
new_card_id = clients['card_id'].max()

new_cards = []

for i in range(len(results)):
    card_id = results.loc[i,'card_id']
    start_date = clients.loc[clients['card_id'] == card_id,'card_start']
    new_card_start_date = pd.to_datetime(start_date.values[0]) + datetime.timedelta(days=days_to_add.pop())
    cancel_date = new_card_start_date + datetime.timedelta(days=days_to_cancel.pop())
    
    
    #update the end date
    clients.loc[clients['card_id'] == card_id,'card_end'] = cancel_date
    
    
    #15% just cancel
    if i < .85*len(results) and new_card_start_date <= pd.to_datetime('2020-12-31'):
        #55% go to Junior2
        #30% go to Classic
        if i<.55*len(results):
            new_card_type = 'Junior2'
        else:
            new_card_type = 'Classic'
        
        new_card_id += 1 
        new_card = clients.loc[clients['card_id'] == card_id].to_dict('records')[0]
        new_card['card_id'] = new_card_id
        new_card['card_type'] = new_card_type
        new_card['card_start'] = new_card_start_date
        new_card['card_end'] = np.nan
        new_card['cc_num'] = fake.credit_card_number()
        new_cards.append(new_card)
    
clients = clients.append(pd.DataFrame(new_cards)) 

In [ ]:
clients = clients.loc[clients['card_start']<pd.to_datetime('2020-12-31')]

In [ ]:
#for each gold member
#change the original card_type 
#then randomly chose a date between now and when the original card was created to sign up for gold
#also if index is in randomly chosen 10% then chose an end date for the gold card also

results = clients.loc[clients['card_type'] == 'Gold'].sample(frac=1).reset_index()

ix_cl = round(len(results)*.6)
ix_j2 = ix_cl+round(len(results)*.1)
ix_cl2 = ix_j2+round(len(results)*.1)
rand_10 = random.sample(range(0, ix_cl2), round(ix_cl2*.25))


new_cards = []
new_card_id = max(clients['card_id'])
curr_ix = 0

days_to_add = list(np.random.normal(600, 150, len(results)))
days_to_cancel = list(np.random.normal(30*3, 25, len(results)))

for ix in [ix_cl,ix_j2,ix_cl2]:
    for i in range(curr_ix,ix):
        card_id = results.loc[i,'card_id']
        disp_id = clients.loc[clients['card_id'] == card_id,'disp_id'].values[0]
        if ix == ix_cl:
            card_type = 'Classic2'
        elif ix == ix_j2:
            card_type = 'Junior2'
        else:
            card_type = 'Classic'
        
        clients.loc[clients['card_id'] == card_id,'card_type'] = card_type
        
        
        created_date = pd.to_datetime(clients.loc[clients['card_id'] == card_id,'card_start'
                             ].values[0])
        new_card_date = created_date + datetime.timedelta(days=days_to_add.pop()) 
        new_cancel_date = np.nan
        
        if i in rand_10[0:15]:
            new_cancel_date = new_card_date + datetime.timedelta(days=days_to_cancel.pop()) 
        
        elif i in rand_10[15:]:
            if created_date != datetime.datetime(2020, 12, 31, 0, 0):
                clients.loc[clients['card_id'] == card_id,'card_end'] = created_date + datetime.timedelta(days=random.randrange((
                    datetime.datetime(2020, 12, 31, 0, 0) - created_date).days)) 

        
        #update cards
        new_card_id += 1 
        new_card = clients.loc[clients['card_id'] == card_id].to_dict('records')[0]
        new_card['card_id'] = new_card_id
        new_card['card_type'] = 'Gold'
        new_card['card_start'] = new_card_date
        new_card['card_end'] = new_cancel_date
        new_card['cc_num'] = fake.credit_card_number()
        new_cards.append(new_card)
        
    curr_ix = ix
clients = clients.append(pd.DataFrame(new_cards)) 

In [ ]:
#add clients staging
pd.DataFrame(clients).to_gbq('finserv_staging.new_clients',if_exists = 'fail')

# Create New Transactions
### If you already have a customers.csv file then you only need to run these cells

In [3]:
clients = pd.read_gbq('select * from finserv_staging.new_clients')
clients['card_start'] = clients['card_start'].dt.tz_localize(None)
clients['card_end'] = pd.to_datetime(clients['card_end']).dt.tz_localize(None)

/Users/leighaj/data_gen/retail_banking_dataset_gen/env/lib/python3.7/site-packages/pandas_gbq/gbq.py:562: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  progress_bar_type=progress_bar_type,


In [5]:
profile_files = [f for f in listdir('profiles/'
                                   ) if isfile(join('profiles/', f))]
profile_files = [join('profiles/', f) 
                 for f in profile_files if f.split('_')[0] in ('adults','young')]

In [6]:
customers_file = open('customers.csv', 'r').readlines() 

In [7]:
headers = ''
for h in ['ssn', 'cc_num', 'first', 'last', 'gender', 'street', \
            'city', 'state', 'zip', 'lat', 'long', 'city_pop', \
            'job', 'dob', 'acct_num', 'profile']:
    headers += h + '|'
headers = headers[:-1]
headers = create_header(headers)

ssn|cc_num|first|last|gender|street|city|state|zip|lat|long|city_pop|job|dob|acct_num|profile|trans_num|trans_date|trans_time|unix_time|category|amt|is_fraud|merchant|merch_lat|merch_long


In [8]:
client = storage.Client()
bucket = client.get_bucket('retail_banking')

In [20]:
def base(a=0):
    counter = 0
    txns = []
    cust_counter = 0

    b=(a+1)*10000
    for i, customer_card in clients[(a*10000):b].iterrows():
        for m in profile_files:
            pro_name = m.split('profiles')[-1]
            pro_name = pro_name[1:]
            parse_index = m.index('profiles') + 9
            m_fraud = m[:parse_index] +'fraud_' + m[parse_index:]
            pro = open(m, 'r').read()
            pro_fraud = open(m_fraud, 'r').read()
            pro_name_fraud = 'fraud_' + pro_name

            curr_profile = pro_name
            curr_fraud_profile = pro_name_fraud
            profile_name = m

            card_id = customer_card['card_id']
            create_d = pd.to_datetime(customer_card['card_start'])

            end_d = pd.to_datetime(customer_card['card_end'])
            if end_d is None or pd.isnull(end_d):
                end_d = pd.to_datetime(date(2021,1,1))
                if (end_d-create_d).days < 10:
                    end_d = pd.to_datetime(date(2021,1,10))
            
            #print(create_d, end_d)
            profile = profile_weights.Profile(pro,create_d,end_d)
            cust = customers_file[[i for i, s in enumerate(customers_file) if customer_card['ssn'] in s][0]]
            t_cust = T_Customer(cust, profile, headers, fake)

            if t_cust.attrs['profile'] == curr_profile:
                is_fraud= 0
                fraud_flag = random.randint(0,100) 
                fraud_dates = []

                merch = pd.read_csv('demographic_data/merchants.csv' , sep='|')
                # decide if we generate fraud or not
                if fraud_flag < 30: #11->25
                    fraud_interval = random.randint(1,1) #7->1
                    inter_val = (end_d-create_d).days-7
                    # rand_interval is the random no of days to be added to start date
                    if inter_val <= 0:
                        inter_val = 1
                    rand_interval = random.randint(1, inter_val)
                    #random start date is selected
                    newstart = create_d + datetime.timedelta(days=rand_interval)
                    # based on the fraud interval , random enddate is selected
                    newend = newstart + datetime.timedelta(days=fraud_interval)
                    # we assume that the fraud window can be between 1 to 7 days #7->1
                    profile = profile_weights.Profile(pro_fraud, newstart, newend)
                    t_cust = T_Customer(cust, profile, headers, fake)
                    is_fraud = 1
                    temp_tx_data = profile.sample_from(is_fraud)
                    fraud_dates = temp_tx_data[3]
                    txns = t_cust.print_trans(temp_tx_data,is_fraud, fraud_dates,merch)
                    #parse_index = m.index('profiles/') + 9
                    #m = m[:parse_index] +'fraud_' + m[parse_index:]

                # we're done with fraud (or didn't do it) but still need regular transactions
                # we pass through our previously selected fraud dates (if any) to filter them
                # out of regular transactions
                profile = profile_weights.Profile(pro, create_d, end_d)
                is_fraud =0
                temp_tx_data = profile.sample_from(is_fraud)
                if txns is not None:
                    txns = txns + t_cust.print_trans(temp_tx_data, is_fraud, fraud_dates, merch)
                else:
                    txns = t_cust.print_trans(temp_tx_data, is_fraud, fraud_dates, merch)


                txns = pd.DataFrame(txns)
                #add in customer id
                txns['card_id'] = customer_card['card_id']

                #if the file isnt more than 5GB then append the transactions to it otherwise create new file
                filename = 'trans_thread_'+str(m)+'_'+str(counter)+'.csv'
                if os.path.getsize(filename)/1000000000 > 5:
                    #upload the file to gcp storage bucket
                    print('added to GCS: ', filename)
                    bucket.blob(filename).upload_from_filename(filename, 'text/csv')
                    counter += 1
                    filename = 'trans_thread_'+str(m)+'_'+str(counter)+'.csv'
                txns.to_csv(filename, mode='a', header=False, sep='|')
                
                loop_counter += 1
                
                
                print(a,'Thread, added ', len(txns),' to the table, for card ', card_id, ' card #: ', 
                      loop_counter, ' in file: ', filename)
                txns = None
    return ('done ', a)

In [21]:
from multiprocessing import Process

procs = []
proc = Process(target=base) 
procs.append(proc)
proc.start()

# instantiating process with arguments
for a in range(12):
    proc = Process(target=base, args=(a,))
    procs.append(proc)
    proc.start()

# complete the processes
for proc in procs:
    proc.join()

2015-11-02 00:00:00 2021-01-01 00:00:00
2015 2021
2020-03-21 00:00:00 2015-11-02 00:00:00 2021-01-01 00:00:002021-01-01 00:00:00
2020
 20152021 
2021
2019-05-27 00:00:002018-09-11 00:00:00 2019-01-22 00:00:00 2018-05-01 00:00:002019-10-05 10:14:20.6516212021-01-01 00:00:002021-01-01 00:00:00   

2021-01-01 00:00:002019-04-27 05:25:29.0254892021-01-01 00:00:0020192018


20192018  2019  2021 202120192021
2021



2015-11-10 00:00:00 2021-01-01 00:00:00
2015 2021
2018-09-27 00:00:002020-02-13 00:00:00 2019-02-01 00:00:002021-01-01 00:00:00  2021-01-01 00:00:00
2021-01-01 00:00:00
2018
 202020192021 
 20212021

2018-09-27 17:15:22.002369 2021-01-01 00:00:00
2018 2021
2015-11-02 00:00:00 2021-01-01 00:00:00
2015 2021
2020-03-21 00:00:00 2021-01-01 00:00:00
2020 2021
2019-05-27 00:00:00 2021-01-01 00:00:00
2019 2021
2018-05-01 00:00:00 2015-11-02 00:00:002019-04-27 05:25:29.025489 
2021-01-01 00:00:002018
 2015 2019
2019-10-05 10:14:20.6516212018-09-11 00:00:0020212020-02-13 00:00:00  2019-01

Process Process-66:
Process Process-75:
Process Process-73:
Traceback (most recent call last):
Process Process-70:
Process Process-74:
Process Process-67:
Process Process-68:
Process Process-72:
Process Process-77:


KeyboardInterrupt: 

Process Process-71:
Process Process-69:
Process Process-76:
Process Process-78:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._

KeyboardInterrupt
  File "/Users/leighaj/data_gen/retail_banking_dataset_gen/env/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 4403, in get_value
    k = self._convert_scalar_indexer(k, kind="getitem")
  File "/Users/leighaj/data_gen/retail_banking_dataset_gen/env/lib/python3.7/site-packages/pandas/core/series.py", line 871, in __getitem__
    result = self.index.get_value(self, key)
KeyboardInterrupt
  File "/Users/leighaj/data_gen/retail_banking_dataset_gen/env/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 4405, in get_value
    return self._engine.get_value(s, k, tz=getattr(series.dtype, "tz", None))
  File "/Users/leighaj/data_gen/retail_banking_dataset_gen/env/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 4379, in get_value
    s = extract_array(series, extract_numpy=True)
  File "/Users/leighaj/data_gen/retail_banking_dataset_gen/env/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 2855, in _convert_scalar_indexer
 